In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/officetel/items/{item_id}"

referer = "https://zigbang.com/home/officetel/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/officetel/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [4]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/officetel/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}

In [8]:
geos_seoul = [
    'wydm',
    'wydq4',
    'wydq5',
    'wydjx',
    'wydjr',
    'wydjq',
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [9]:
from itertools import islice

len(items)

sliceItem = dict(islice(items.items(), 5000))

len(sliceItem)

5000

In [10]:
print(sliceItem)

{'29199566': {}, '28258611': {}, '28866748': {}, '29277957': {}, '29279747': {}, '29291596': {}, '29270504': {}, '29196976': {}, '29180889': {}, '29196658': {}, '29243498': {}, '29170297': {}, '29281626': {}, '29255090': {}, '29274787': {}, '29174480': {}, '29241785': {}, '29226497': {}, '29230408': {}, '29133085': {}, '29272277': {}, '29256275': {}, '29021923': {}, '29253227': {}, '29212054': {}, '29277325': {}, '29122570': {}, '29227160': {}, '29116218': {}, '29280314': {}, '29282052': {}, '29211727': {}, '29296340': {}, '29092846': {}, '29236984': {}, '29287067': {}, '29131891': {}, '29207325': {}, '29294386': {}, '29266929': {}, '29067824': {}, '29259649': {}, '29274252': {}, '29052247': {}, '29230521': {}, '29172647': {}, '28930309': {}, '29195588': {}, '29247397': {}, '29296641': {}, '29291745': {}, '29211842': {}, '29206535': {}, '29228704': {}, '29227050': {}, '29242872': {}, '29244051': {}, '29281323': {}, '29206491': {}, '29232127': {}, '29270403': {}, '28997533': {}, '292910

In [11]:
result = describe_room_list(sliceItem)

In [12]:
len(result)

4833

In [13]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '2층',
  'address1': '서울시 동작구 사당동',
  'address2': '303-83',
  'address3': None,
  'agent_address1': '서울특별시 동작구 동작대로27길 28 (사당동 )',
  'agent_comment': '',
  'agent_email': 'model603@naver.com',
  'agent_lat': 37.4876174293583,
  'agent_lng': 126.98042967964902,
  'agent_local1': '서울시',
  'agent_local2': '동작구',
  'agent_mobile': '010-5408-1413',
  'agent_name': '리치공인중개사사무소(최현승)',
  'agent_no': 0,
  'agent_phone': '02-532-6966',
  'bjd_code': '',
  'bonbun_code': '303',
  'bubun_code': '83',
  'building': None,
  'building_type': '-',
  'contract': '서울특별시',
  'deposit': 18000,
  'description': '💖올리모델링!\r\n'
                 '\r\n'
                 '💖전세자금대출가능!\r\n'
                 '\r\n'
                 '💖보증금 조절가능!\r\n'
                 '\r\n'
                 '💖주차가능!\r\n'
                 '\r\n'
                 '💖주변 편의시설 다수인접!\n'
                 '21.10.25에 직방에 등록된 매물입니다.',
  'description_og': '&#128150;올리모델링!\r\n'
                    '\r\n'
                    '&#128150;전세자

In [14]:
filtered_result = export_items(result)

In [15]:
len(filtered_result)

3076

In [16]:
def get_string_image(item):
    image = ''
    
    for list in item:
        image += list['url'] + '?w=400&h=300&q=70&a=1,'
        
    return image[:-1]

In [17]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [18]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [19]:
def get_agent_lat(item):
    agent_lat = str(item['agent_lat'])[0:9]
    
    return agent_lat

def get_agent_lng(item):
    agent_lng = str(item['agent_lng'])[0:10]
    
    return agent_lng

In [20]:
geo_based_dict = {}

In [21]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)

    item['lat'] = get_location_lat(item)
    item['lng'] = get_location_lng(item)
    item['image'] = get_string_image(item['images'])
    item['agent_lat'] = get_agent_lat(item)
    item['agent_lng'] = get_agent_lng(item)    

In [22]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '경기도 하남시 덕풍동 330-1 ',
  'address1': '경기도 하남시 덕풍동',
  'address2': '330-1',
  'address3': None,
  'agent_address1': '경기도 하남시 하남대로855번길 16 (덕풍동)',
  'agent_email': 'minseo9179@naver.com',
  'agent_lat': '37.542727',
  'agent_lng': '127.202663',
  'agent_mobile': '010-9677-9179',
  'agent_name': '햇살좋은집공인중개사사무소(김민서)',
  'agent_phone': '031-795-8944',
  'building_type': '-',
  'deposit': 30000,
  'description': '- 신축2년 투룸(방2, 화장실1) 전세입니다.\r\n'
                 '- 거실과 부엌이 넓어 공간활용하기 좋습니다. \r\n'
                 '- 남향이라 햇빛이 잘들어 밝고 환한 집입니다.\r\n'
                 '- 방2개 모두 공간 좋아요.\r\n'
                 '- 깔끔한 집이라 신혼부부에게 강추합니다.\r\n'
                 '- 에어컨, 세탁기 설치되어 있습니다.\r\n'
                 '- 하남시청역 도보5분거리에 위치해있습니다.\r\n'
                 '- 버스정류장 도보 3분거리입니다.\r\n'
                 '- 주차 가능합니다.\r\n'
                 '- 전세자금대출 가능합니다.\n'
                 '21.11.01에 직방에 등록된 매물입니다.',
  'elevator': '있음',
  'floor': '7층',
  'floor_all': '7층',
  'image': 'https://ic.zigbang.com/ic/items/292

In [23]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [24]:
with open('officetel.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'image',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [1]:
import pymysql
import csv
conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8mb4')
curs = conn.cursor()

sql = """insert into officetel (address, agent_address, agent_email, agent_lat, agent_lng,
                                agent_mobile, agent_name, agent_phone, building_type, deposit, 
                                description, elevator, floor, floor_all, image,
                                lat, lng, local1, local2, local3,
                                manage_cost, manage_cost_inc, movein_date, near_subways, options,
                                parking, pets, room_direction, room_type, sales_type,
                                service_type, size, size_m2, title, updated_at,
                                user_intro, user_name, url)
                                values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                        %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                        %s,%s,%s,%s,%s,%s,%s,%s)"""

f = open('officetel.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

next(rd)

for line in rd:
    curs.execute(sql, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 
                       line[11], line[12], line[13], line[14], line[15], line[16], line[17], line[18], line[19], line[20], 
                       line[21], line[22], line[23], line[24], line[25], line[26], line[27], line[28], line[29], line[30], 
                       line[31], line[32], line[33], line[34], line[35], line[36], line[37]))
conn.commit()
conn.close()
f.close()